# Prepare DF

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

module_path

'/home/tjanicek/thesis/thoth-issue-predictor'

In [2]:
import graphviz
from pandas_profiling import ProfileReport
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from thoth_issue_predictor.utils.grid_search import grid_search

from thoth_issue_predictor.utils.preprocessing import (
    prepare_df,
    create_python_version_packege_df,
)

In [3]:
FILE_NAME = "../datasets/dataset.zip"
inspections_df = prepare_df(FILE_NAME)
exit_codes = inspections_df["exit_code"]

INFO:thoth.report_processing.components.inspection:Retrieving dataset at path... /home/tjanicek/thesis/thoth-issue-predictor/thoth_issue_predictor/notebooks/inspections
INFO:thoth.report_processing.components.inspection:Considering inspection ID... inspection-tf-dm-six-d9316be5
INFO:thoth.report_processing.components.inspection:Considering inspection ID inspection-tf-dm-six-d9316be5.Number 0
INFO:thoth.report_processing.components.inspection:Considering inspection ID... inspection-tf-dm-rw-4cf3afae
INFO:thoth.report_processing.components.inspection:Considering inspection ID inspection-tf-dm-rw-4cf3afae.Number 0
INFO:thoth.report_processing.components.inspection:Considering inspection ID... inspection-tf-dm-np-605fcb35
INFO:thoth.report_processing.components.inspection:Considering inspection ID inspection-tf-dm-np-605fcb35.No files retrieved due to the following error: [Errno 2] No such file or directory: '/home/tjanicek/thesis/thoth-issue-predictor/thoth_issue_predictor/notebooks/inspe

In [4]:
(
    issues_df,
    _,
    python_indexes,
) = create_python_version_packege_df(inspections_df=inspections_df)
issues_df["exit_code"] = exit_codes.astype("int")
issues_df = issues_df.loc[
    (issues_df.loc[:, issues_df.columns != "exit_code"] != 0).any(axis=1)
]
issues_df = issues_df.drop_duplicates()

In [5]:
python_indexes

['unknown', 'pypi-org', 'aicoe', 'pypi']

In [6]:
print(f"Length of DF is {len(issues_df)}")
issues_df.head(5)

Length of DF is 424


,python,tensorflow_major,tensorflow_minor,tensorflow_patch,tensorflow_index,absl-py_major,absl-py_minor,absl-py_patch,absl-py_index,astor_major,...,keras_index,pyyaml_major,pyyaml_minor,pyyaml_patch,pyyaml_index,tensorflow-cpu_major,tensorflow-cpu_minor,tensorflow-cpu_patch,tensorflow-cpu_index,exit_code
0,36,2,1,0,1,0,8,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,36,2,1,0,1,0,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,36,2,1,0,1,0,8,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,36,1,11,0,1,0,8,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424 entries, 0 to 465
Columns: 174 entries, python to exit_code
dtypes: int64(174)
memory usage: 579.7 KB


In [9]:
profile = ProfileReport(issues_df, title="Tensorflow issues report")
profile.to_file("thoth_issue_report.html")

Summarize dataset:   0%|          | 0/188 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Failed inspections

In [10]:
failed_inspections = issues_df[issues_df["exit_code"] == 1]
print(f"Number of failed inspections: {len(failed_inspections)}")
failed_inspections.head()

Number of failed inspections: 102


,python,tensorflow_major,tensorflow_minor,tensorflow_patch,tensorflow_index,absl-py_major,absl-py_minor,absl-py_patch,absl-py_index,astor_major,...,keras_index,pyyaml_major,pyyaml_minor,pyyaml_patch,pyyaml_index,tensorflow-cpu_major,tensorflow-cpu_minor,tensorflow-cpu_patch,tensorflow-cpu_index,exit_code
362,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,1
363,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,1
364,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,1
365,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,1
366,36,2,1,0,1,0,9,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
issue_x, issue_y = issues_df.drop("exit_code", axis=1), issues_df

# Decision Trees

In [12]:
best_params = grid_search(issue_x, issue_y["exit_code"])

# Tuning hyper-parameters for precision



/home/tjanicek/.local/share/virtualenvs/thoth-issue-predictor-jSoEZjm6/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tjanicek/.local/share/virtualenvs/thoth-issue-predictor-jSoEZjm6/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tjanicek/.local/share/virtualenvs/thoth-issue-predictor-jSoEZjm6/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Best parameters set found on development set:

{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 5, 'random_state': 20, 'splitter': 'random'}

Grid scores on development set:

0.965 (+/-0.097) for {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 10, 'splitter': 'best'}
0.955 (+/-0.067) for {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 10, 'splitter': 'random'}
0.977 (+/-0.038) for {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 20, 'splitter': 'best'}
0.977 (+/-0.064) for {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 20, 'splitter': 'random'}
0.965 (+/-0.097) for {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_s

In [13]:
clf = DecisionTreeClassifier(**best_params)
cross_val_score(clf, issue_x, issue_y["exit_code"], cv=10)

array([1.        , 0.95348837, 1.        , 1.        , 0.95238095,
       0.92857143, 0.97619048, 0.95238095, 0.97619048, 0.97619048])

In [14]:
clf.fit(issue_x, issue_y["exit_code"])

DecisionTreeClassifier(max_depth=8, min_samples_split=5, random_state=20,
                       splitter='random')

In [15]:
feature_names = issue_x.columns
target_names = ["successful", "failed"]
dot_data = export_graphviz(
    clf,
    out_file=None,
    feature_names=feature_names,
    class_names=target_names,
    filled=True,
    rounded=True,
    special_characters=True,
)
graph = graphviz.Source(dot_data)
graph.render("ThothIssuePredictor")

'ThothIssuePredictor.pdf'